In [1]:
import polars as pl
from polars import col as c
import altair as alt
from camminapy.plot import altair_theme

altair_theme()

In [2]:
df = pl.read_parquet("../data/data.parquet")

In [8]:
df.filter(pl.col("Rank") == 1).sort("Date").write_csv("../data/worldrecords.csv")